In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

from bs4 import BeautifulSoup
import ast
import time
import random
import requests
import re
import json
import ast

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [22]:
df = pd.read_csv('new_sample_data.csv', encoding='ISO-8859-1')

In [23]:
# Support Functions

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def contains_phone_num(string):
    regexp = re.compile(r'[2-9]\d{2}-\d{3}-\d{4}$')
    if regexp.search(string):
        return True
    else:
        return False

def str_to_dict(string):
    string=string.replace('[','')
    string=string.replace(']','')
    newDict=ast.literal_eval(string)
    return newDict

def bullets_to_list(df):
    bullets=df[['ASIN','Bullet Points']]
    delete=['[',']',"\""]
    for index, row in bullets.iterrows():
        for char in delete:
            bullets.loc[index, 'Bullet Points'] = row['Bullet Points'].replace(char,'')
        bullets.loc[index, 'Bullet Points'] = row['Bullet Points'].split('\', ')
    return bullets

In [24]:
def main(df):
    start = time.time()

    # Collect sub_dfs and merge
    titles=analyze_titles(df)
    descriptions=analyze_descriptions(df)
    media=analyze_media(df)
    reviews=analyze_ratings_and_reviews(df)
    
    df=pd.merge(titles,descriptions,on='ASIN')
    df=pd.merge(df,media,on='ASIN')
    df=pd.merge(df,reviews,on='ASIN')

    # Top-Level Analysis
    results = df[['ASIN','Product Name_x']].copy()
    results.rename(columns={'Product Name_x':'Product Name'}, inplace=True)
    results = results.assign(**{'TITLE: Capitalize First Letters of Each Word': 0,
                                'TITLE: Do not use ALL CAPS in title': 0,
                                'TITLE: Do not capitalize conjunctions/articles': 0,
                                'TITLE: Do not use single quotes/apostrophes': 0,
                                'TITLE: Do not SEO-adverse characters': 0,
                                'TITLE: Do not include Type 1 ASCII symbols': 0,
                                'TITLE: Do not include promotional phrases': 0,
                                "TITLE: Do not include '$' sign": 0,
                                'Title Total Grade': 0,
                                
                                'DESCRIPTION: 250+ Characters': 0,
                                'DESCRIPTION: 500+ Characters': 0,
                                'DESCRIPTION: 1,500+ Characters': 0,
                                'DESCRIPTION: 1,600-2,100 Characters': 0,
                                'DESCRIPTION: Do not include quotes or testimonials': 0,
                                'DESCRIPTION: Do not include HTML code': 0,
                                'DESCRIPTION: Do not include price or condition information': 0,
                                'DESCRIPTION: Do not include shipping information': 0,
                                'DESCRIPTION: Do not include contact information (phone, email, website, etc.)': 0,
                                'DESCRIPTION: Include 3+ bullet points': 0,
                                'DESCRIPTION: Include 4+ bullet points': 0,
                                'DESCRIPTION: Include 5+ bullet points': 0,
                                'DESCRIPTION: Capitalize each bullet point': 0,
                                'Description Total Grade': 0,
                               
                                'MEDIA: Include 5+ Images': 0,
                                'MEDIA: Include 6+ Images': 0,
                                'MEDIA: Include 7+ Images': 0,
                                'MEDIA: Include 8+ Images': 0,
                                'MEDIA: Include <10 Media Files': 0,
                                'MEDIA: Include 1+ Videos': 0,
                                'MEDIA: Use images of 1,000 x 1,000 resolution': 0,
                                'Media Total Grade': 0,
                               
                                'PRODUCT: 4.5+ star rating': 0,
                                'PRODUCT: 4.0+ star rating': 0,
                                'PRODUCT: 3.5+ star rating': 0,
                                'PRODUCT: 3.0+ star rating': 0,
                                'PRODUCT: 10+ reviews': 0,
                                'PRODUCT: 25+ reviews': 0,
                                'PRODUCT: 50+ reviews': 0,
                                'PRODUCT: 100+ reviews': 0,
                                'PRODUCT: 250+ reviews': 0,
                                'Product Total Grade': 0})
    for index, row in df.iterrows():
        
        ### Titles ###
        
        # Capitalization
        if row['num_lower_case'] <= 1:
            results.loc[index, 'TITLE: Capitalize First Letters of Each Word'] = 5
        if row['num_all_caps'] < 1:
            results.loc[index, 'TITLE: Do not use ALL CAPS in title'] = 5
        if row['num_incorrect_caps'] < 1:
            results.loc[index, 'TITLE: Do not capitalize conjunctions/articles'] = 5

        # Numbers and symbols
        if row['contains_single_quote'] == False:
            results.loc[index, 'TITLE: Do not use single quotes/apostrophes'] = 5
        if row['contains_SEO_adverse_chars'] == False:
            results.loc[index, 'TITLE: Do not SEO-adverse characters'] = 5
        if row['contains_ASCII'] == False:
            results.loc[index, 'TITLE: Do not include Type 1 ASCII symbols'] = 5
        
        # Do not include
        if row['contains_promo_phrase'] == False:
            results.loc[index, 'TITLE: Do not include promotional phrases'] = 12
        if row['contains_$'] == False:
            results.loc[index, "TITLE: Do not include '$' sign"] = 8
        
        
        ### Descriptions ###
        
        # Character counts
        if row['descr_char_count'] >= 250:
            results.loc[index, 'DESCRIPTION: 250+ Characters'] = 8
        if row['descr_char_count'] >= 500:
            results.loc[index, 'DESCRIPTION: 500+ Characters'] = 7
        if row['descr_char_count'] >= 1500:
            results.loc[index, 'DESCRIPTION: 1,500+ Characters'] = 5
        if (row['descr_char_count'] >= 1600 and row['descr_char_count'] <= 2100):
            results.loc[index, 'DESCRIPTION: 1,600-2,100 Characters'] = 5
        
        # Do not include
        if row['contains_quotes'] == False:
            results.loc[index, 'DESCRIPTION: Do not include quotes or testimonials'] = 5
        if row['contains_html'] == False:
            results.loc[index, 'DESCRIPTION: Do not include HTML code'] = 5
        if row['contains_price_condition_info'] == False:
            results.loc[index, 'DESCRIPTION: Do not include price or condition information'] = 5
        if row['contains_shipping_info'] == False:
            results.loc[index, 'DESCRIPTION: Do not include shipping information'] = 5
        if row['contains_contact_info'] == False:
            results.loc[index, 'DESCRIPTION: Do not include contact information (phone, email, website, etc.)'] = 15
        
        # Bullets
        if row['3+ Bullets'] == True:
            results.loc[index, 'DESCRIPTION: Include 3+ bullet points'] = 5
        if row['4+ Bullets'] == True:
            results.loc[index, 'DESCRIPTION: Include 4+ bullet points'] = 10
        if row['5+ Bullets'] == True:
            results.loc[index, 'DESCRIPTION: Include 5+ bullet points'] = 10
        if row['num_lower_case_bullets'] < 1:
            results.loc[index, 'DESCRIPTION: Capitalize each bullet point'] = 10
        
        ### Media ###
        
        # Images and videos
        if row['5+_images'] == True:
            results.loc[index, 'MEDIA: Include 5+ Images'] = 30
        if row['6+_images'] == True:
            results.loc[index, 'MEDIA: Include 6+ Images'] = 15
        if row['7+_images'] == True:
            results.loc[index, 'MEDIA: Include 7+ Images'] = 10
        if row['8+_images'] == True:
            results.loc[index, 'MEDIA: Include 8+ Images'] = 5
        if row['<_10_media'] == True:
            results.loc[index, 'MEDIA: Include <10 Media Files'] = 10
        if row['num_videos'] >= 1:
            results.loc[index, 'MEDIA: Include 1+ Videos'] = 10
        if row['low_qual_images'] < 1:
            results.loc[index, 'MEDIA: Use images of 1,000 x 1,000 resolution'] = 20
            
        # Ratings, reviews and category rankings
        
        # Product ratings
        if row['Rating'] >= 4.5:
            results.loc[index, 'PRODUCT: 4.5+ star rating'] = 20
        if row['Rating'] >= 4.0:
            results.loc[index, 'PRODUCT: 4.0+ star rating'] = 30
        if row['Rating'] >= 3.5:
            results.loc[index, 'PRODUCT: 3.5+ star rating'] = 15
        if row['Rating'] >= 3.0:
            results.loc[index, 'PRODUCT: 3.0+ star rating'] = 10
        
        # Product reviews
        reviews = int(row['Reviews'].replace(',',''))
        if reviews >= 10:
            results.loc[index, 'PRODUCT: 10+ reviews'] = 5
        if reviews >= 25:
            results.loc[index, 'PRODUCT: 25+ reviews'] = 5
        if reviews >= 50:
            results.loc[index, 'PRODUCT: 50+ reviews'] = 5
        if reviews >= 100:
            results.loc[index, 'PRODUCT: 100+ reviews'] = 5
        if reviews >= 250:
            results.loc[index, 'PRODUCT: 250+ reviews'] = 5
        
    totals = ['Title','Description','Media','Product']
    for total in totals:
        cols = results.columns.str.startswith(str(total).upper())
        results[str(total + ' Total Grade')] = results.iloc[:, cols].sum(axis=1)    
    
    results.to_csv('delete.csv')
    end = time.time()
    print('Run time (sec):', str(end - start))
    print(str(len(df.index)),' ASINs')
    
    return results

In [25]:
%%notify
main(df)

ft
bar-Sized
mini


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Run time (sec): 8.898626804351807
32  ASINs


,ASIN,Product Name,TITLE: Capitalize First Letters of Each Word,TITLE: Do not use ALL CAPS in title,TITLE: Do not capitalize conjunctions/articles,TITLE: Do not use single quotes/apostrophes,TITLE: Do not SEO-adverse characters,TITLE: Do not include Type 1 ASCII symbols,TITLE: Do not include promotional phrases,TITLE: Do not include '$' sign,...,PRODUCT: 4.5+ star rating,PRODUCT: 4.0+ star rating,PRODUCT: 3.5+ star rating,PRODUCT: 3.0+ star rating,PRODUCT: 10+ reviews,PRODUCT: 25+ reviews,PRODUCT: 50+ reviews,PRODUCT: 100+ reviews,PRODUCT: 250+ reviews,Product Total Grade
0,B0194WDVHI,"Anker PowerCore 10000, One Great Deal of The S...",5,5,0,0,5,5,0,8,...,20,30,15,10,5,5,5,5,5,100
1,B00X5RV14Y,Portable Charger Anker PowerCore 20100mAh - Ul...,5,0,5,0,5,5,12,8,...,20,30,15,10,5,5,5,5,5,100
2,B00Z9QVE4Q,Anker PowerCore 13000 ? Portable Charger - Com...,5,5,5,5,0,5,12,8,...,20,30,15,10,5,5,5,5,5,100
3,B016XTADG2,Anker Soundcore ! [Bluetooth Speaker with Tren...,5,5,5,5,0,5,0,0,...,20,30,15,10,5,5,5,5,5,100
4,B071YMZ4LD,"Anker's Elite USB Charger, Dual Port 24W Wall ...",5,0,5,0,0,5,12,8,...,20,30,15,10,5,5,5,5,5,100
5,B07DDHC5MV,"Anker Powerhouse 200, 200Wh/57600mAh Portable ...",5,0,5,5,0,5,12,8,...,0,30,15,10,5,0,0,0,0,60
6,B07DBXZZN3,"Anker Wireless Charger, 10W Wireless Charging ...",5,5,5,5,0,5,0,8,...,0,30,15,10,5,5,5,5,5,80
7,B01HXYNYUS,[3 Pack: 3ft/6ft/10ft] Anker Powerline Lightni...,5,5,5,5,0,5,12,8,...,0,30,15,10,5,5,5,5,5,80
8,B07D9C8NP2,Anker 6ft Premium Nylon Lightning Cable [2-Pac...,5,5,5,5,0,5,12,8,...,0,30,15,10,5,5,5,5,5,80
9,B07PYG4JZR,"Anker PowerCore Slim 10000 PD, 10000mAh Portab...",5,0,5,5,0,5,12,8,...,20,30,15,10,5,0,0,0,0,80


<IPython.core.display.Javascript object>

In [6]:
### Timing functions ###

start = time.time()
analyze_titles(df)
end = time.time()

print('analyze_titles run time (sec):', str(end - start))
print()

start = time.time()
analyze_descriptions(df)
end = time.time()
print('analyze_descriptions run time (sec):', str(end - start))
print()

start = time.time()
analyze_media(df)
end = time.time()
print('analyze_media run time (sec):', str(end - start))
print()

start = time.time()
analyze_ratings_and_reviews(df)
end = time.time()
print('analyze_ratings_and_reviews run time (sec):', str(end - start))
print()


NameError: name 'analyze_titles' is not defined

#### Analyze Titles

In [7]:
def analyze_titles(df):
    
    product_titles = df[['ASIN','Product Name']].copy()
    product_titles['Title: # Characters'] = product_titles['Product Name'].str.len()
    product_titles['title_list'] = product_titles['Product Name'].str.split(' ')
    product_titles = product_titles.assign(**{'title_keywords': '',
                                              'num_lower_case': 0,
                                              'num_incorrect_caps': 0,
                                              'num_all_caps': 0,
                                              #'contains_numbers': False,
                                              'contains_ASCII': False,
                                              'contains_SEO_adverse_chars': False,
                                              'contains_promo_phrase': False,
                                              'contains_single_quote': False,
                                              'contains_$': False})
    ignore=['with','of','and','or','to','the','a','an','at','for','in','over','on','iPhone','iPad']
    for index, row in product_titles.iterrows():

        # Character counts
        num_chars=row['Title: # Characters']
        product_titles['60+ Characters'] = num_chars >= 60
        
        # Checking for promo phrases appearing in full title
        promo_phrases = ['Bargain','Free Shipping','Discount','Promotion','Promo','Hot Deal',
                      'Great Deal','Hot item','Best Price','Best Seller','Better','Trending','Trendy']
        if any(phrase.lower() in str(row['Product Name']).lower() for phrase in promo_phrases):
            product_titles.loc[index, 'contains_promo_phrase'] = True
        
        # Checking for single-quote/apostrophes?
        if '\'' in str(row['Product Name']):
            product_titles.loc[index, 'contains_single_quote'] = True
            
        # Checking for Type 1 ASCII
        is_ASCII = (lambda s: len(s) != len(s.encode()))
        if is_ASCII(str(row['Product Name'])):
            product_titles.loc[index, 'contains_ASCII'] = True

        # Checking for SEO-adverse characters
        badChars=['~','!','*','$','?','_','~','{','}','[',']','#','&lt;','>','|','*',';','/','^','¬','¦']
        if any(char in str(row['Product Name']) for char in badChars):
            product_titles.loc[index, 'contains_SEO_adverse_chars'] = True

        
        # Checking for word-by-word violations
        title_keywords = []
        for word in row['title_list']:
            word=word.replace(',','')
            if word.lower() not in (item.lower() for item in ignore):
                title_keywords.append(word)
            
            # Checking for words not starting with upper case
            try:
                if str(word)[0].islower() and str(word).lower() not in (word.lower() for word in ignore):
                    print(str(word))
                    product_titles.loc[index, 'num_lower_case'] +=1
            except:
                print('error line 20: ' + str(word))

            # Checking for any all caps words
            if word.isupper() and len(word) >3:
                product_titles.loc[index,'num_all_caps'] +=1
            
            # Checking for mis-capitalized conjunctions & articles
            try:
                if str(word)[0].isupper() and str(word).lower() in ignore:
                    product_titles.loc[index,'num_incorrect_caps'] +=1
            except:
                print('error line 43: ' + word)
        
            """
            # Do not include promotional/subjective phrases
            global promo_phrases
            promo_phrases = ['Bargain','Free Shipping','Discount','Promotion','Promo','Hot Deal',
                          'Great Deal','Hot item','Best Price','Best Seller','Better','Trending','Trendy']
            if str(word).lower() in (phrase.lower() for phrase in promo_phrases):
                print(str(word))
                product_titles.loc[index, 'contains_promo_phase'] = True
            """
            
            # Checking for dollar sign $
            if '$' in str(word):
                product_titles.loc[index, 'contains_$'] = True

        #product_titles.loc[index, 'title_keywords'] = title_keywords
        
    del product_titles['title_list']
    return product_titles
analyze_titles(df.head(10))

ft


,ASIN,Product Name,Title: # Characters,title_keywords,num_lower_case,num_incorrect_caps,num_all_caps,contains_ASCII,contains_SEO_adverse_chars,contains_promo_phrase,contains_single_quote,contains_$,60+ Characters
0,B0194WDVHI,"Anker PowerCore 10000, One Great Deal of The S...",207,,0,1,0,False,False,True,True,False,True
1,B00X5RV14Y,Portable Charger Anker PowerCore 20100mAh - Ul...,191,,0,0,1,False,False,False,True,False,True
2,B00Z9QVE4Q,Anker PowerCore 13000 ? Portable Charger - Com...,188,,0,0,0,False,True,False,False,False,True
3,B016XTADG2,Anker Soundcore ! [Bluetooth Speaker with Tren...,209,,1,0,0,False,True,True,False,True,True
4,B071YMZ4LD,"Anker's Elite USB Charger, Dual Port 24W Wall ...",190,,0,0,1,False,True,False,True,False,True
5,B07DDHC5MV,"Anker Powerhouse 200, 200Wh/57600mAh Portable ...",196,,0,0,2,False,True,False,False,False,True
6,B07DBXZZN3,"Anker Wireless Charger, 10W Wireless Charging ...",206,,0,0,0,False,True,True,False,False,True
7,B01HXYNYUS,[3 Pack: 3ft/6ft/10ft] Anker Powerline Lightni...,187,,0,0,0,False,True,False,False,False,True
8,B07D9C8NP2,Anker 6ft Premium Nylon Lightning Cable [2-Pac...,183,,0,0,0,False,True,False,False,False,True
9,B07PYG4JZR,"Anker PowerCore Slim 10000 PD, 10000mAh Portab...",183,,0,0,4,False,True,False,False,False,True


#### Analyze Descriptions

In [8]:
def analyze_descriptions(df):
    
    df = df[['ASIN','Product Name','Product Description','Bullet Points']].copy()
    bullets=bullets_to_list(df)
    del df['Bullet Points']
    df = pd.merge(df, bullets, on='ASIN')

    df['num_bullets'] = df['Bullet Points'].str.len()
    df['descr_char_count']  = df['Product Description'].str.len()
    df = df.assign(**{'contains_quotes': False,
                      'contains_price_condition_info': False,
                      'contains_shipping_info': False,
                      'contains_contact_info': False,
                      'contains_html': False,
                      'num_lower_case_bullets': 0})
    
    for index, row in df.iterrows():
        description=str(row['Product Description'])
        
        # Checking for quotes
        if "\"" in description:
            df.loc[index, 'contains_quotes'] = True
    
        # Check for HTML/JS in descriptions
        if bool(BeautifulSoup(description, "html.parser").find()):
            df.loc[index, 'contains_html'] = True
        
        # Check for price/condition details
        flags=['Mint Condition','Lowest price','Great Offer','Best Price',
               'Great Condition','Good Condition','Lowest Offer']
        if any(flag in description for flag in flags):
            df.loc[index, 'contains_price_condition_info'] = True
        
        # Check for shipping offers/details
        flags=['Free Shipping','Fast Shipping','Day Shipping']
        if any(flag in description for flag in flags):
            df.loc[index, 'contains_shipping_info'] = True
        
        # Check for contact info (phone #s, emails, websites, etc.)
        flags=['@','.com','.org']
        if any(flag in description for flag in flags) or contains_phone_num(description):
            df.loc[index, 'contains_contact_info'] = True

            
        ### Bullets ###
        # Checking that bullets start with upper case
        try:
            for bullet in row['Bullet Points']:
                bullet = str(bullet).replace('\'','')
                if (str(bullet)[0].isupper() and not hasNumbers(str(bullet))):
                    df.loc[index, 'num_lower_case_bullets'] +=1
        except:
            print('No bullets found.')
    
    df['3+ Bullets'] = df['num_bullets']>=3
    df['4+ Bullets'] = df['num_bullets']>=4
    df['5+ Bullets'] = df['num_bullets']>=5
    
    del df['Product Name']
    
    return df

#### Analyze Media

In [9]:
def analyze_media(df):
    df = df[['ASIN','Product Name','Images','Videos']].copy()
    df = df.assign(**{'num_images': 0,
                      'num_videos': 0,
                      'low_qual_images': 0,
                      'high_whitespace_images': 0})
    for index, row in df.iterrows():
        try:
            images = str_to_dict(str(row['Images']))
            df.loc[index, 'num_images'] = len(images)
            for image in images:
                # Num images > 1000x1000px
                if image['width'] < 1000 and image['height'] < 1000:
                    df.loc[index, 'low_qual_images'] +=1
                # High whitespace images
                if round(image['whitespace'],2) < 0.85:
                    df.loc[index, 'high_whitespace_images'] +=1
        except:
            continue
            #print('No Images found.')
        else:
            continue
            #print()
        
        # Has >= 1 video
        try:
            videos = str_to_dict(str(row['Videos']))
        except:
            continue
            #print('No videos found.')
        else:
            if len(videos)>0:
                df.loc[index, 'num_videos'] = len(videos)
                print('Num videos:',len(videos))
        
    # Image thresholds
    cols=['5+_images','6+_images','7+_images','8+_images']
    for col in cols:
        df[str(col)] = df['num_images'] >= int(str(col)[0])
    df['<_10_media'] = (df['num_images'] + df['num_videos']) < 10
    
    del df['Product Name']
    return df

#### Analyze Product Ratings & Reviews

In [27]:
def analyze_ratings_and_reviews(df):
    
    ### Ratings & Reviews ###
    df = df[['ASIN','Product Name','Rating','Reviews','Product Rankings']].copy()
    
    ### Category Rankings ###
    
    category_rankings = df[['ASIN','Product Rankings']].copy()
    category_rankings['Product Rankings'] = category_rankings['Product Rankings'].replace({'\"': '\''}, regex=True)

    characters=['#','[',']']
    for char in characters:
        category_rankings['Product Rankings'] = category_rankings['Product Rankings'].str.replace(str(char),'')
    
    category_rankings = category_rankings['Product Rankings'].str.split("', '", expand=True)
    category_rankings = category_rankings.replace('\'', '', regex=True)
    category_rankings = pd.concat([df['ASIN'], category_rankings], axis=1)
    
    for col in list(category_rankings):
        if str(col) == 'ASIN' or str(col).startswith('Category Ranking '):
            continue
        else:
            category_rankings.rename({col: 'Category Ranking ' + str(col)}, axis='columns', inplace=True)

    cols = list(category_rankings)[2:]
    
    all_rankings = category_rankings[['ASIN','Category Ranking 0']].copy()
    
    for col in cols:
        loop_df = pd.concat([category_rankings['ASIN'], category_rankings[str(col)]], axis=1)
        loop_df.rename(columns={str(col): 'Category Ranking 0'}, inplace=True)
        all_rankings = pd.concat([all_rankings, loop_df], ignore_index = True) #.reset_index(drop=True)

    all_rankings.sort_values('ASIN',inplace=True)
    all_rankings.dropna(inplace=True)
    all_rankings.rename(columns={'Category Ranking 0':'Category Ranking'},inplace=True)
    all_rankings[['Ranking','Category']] = all_rankings['Category Ranking'].str.split(' in ', expand=True)
    del all_rankings['Category Ranking']
    
    all_rankings = all_rankings.groupby('ASIN').Ranking.apply(list).apply(pd.Series).fillna('').reset_index()
    num_rankings = 0
    for col in list(all_rankings)[1:]:
        all_rankings.rename(columns={col:'Category Ranking #' + str(col)}, inplace=True)
        try:
            n = int(col) +1
            if n > num_rankings:
                num_rankings = n
        except:
            continue

    df = pd.merge(df, all_rankings, on='ASIN')
    return df
analyze_ratings_and_reviews(df.head())

,ASIN,Product Name,Rating,Reviews,Product Rankings,Category Ranking #0,Category Ranking #1,Category Ranking #2
0,B0194WDVHI,"Anker PowerCore 10000, One Great Deal of The S...",4.5,"11,086","['#388 in Cell Phones & Accessories', '#1 in U...",388,1,
1,B00X5RV14Y,Portable Charger Anker PowerCore 20100mAh - Ul...,4.5,"15,866","['#5,025 in Computers & Accessories', '#179 in...","5,025",179,
2,B00Z9QVE4Q,Anker PowerCore 13000 ? Portable Charger - Com...,4.6,"7,029","['#202 in Cell Phones & Accessories', '#9 in P...",202,9,
3,B016XTADG2,Anker Soundcore ! [Bluetooth Speaker with Tren...,4.5,"13,415","['#108 in Cell Phones & Accessories', '#3 in P...",108,3,39
4,B071YMZ4LD,"Anker's Elite USB Charger, Dual Port 24W Wall ...",4.5,"5,208","['#27 in Cell Phones & Accessories', '#1 in Ce...",27,1,


**# Product Reviews (~ >=50)**

In [11]:
def analyze_product_reviews(df):
    review_counts = df[['ASIN','Reviews']].copy()
    review_count_violations = review_counts.loc[review_counts['Reviews']<50]
    review_count_violations['violation'] = '< 50 Product Reviews'
    
    return review_count_violations

**No Product Ratings or Ratings < 4.0 Stars**

In [12]:
def analyze_product_ratings(df):
    product_ratings = df[['ASIN','Rating']].copy()
    
    no_ratings = product_ratings.loc[product_ratings['Rating']=='No Ratings']
    no_ratings['violation'] = 'No Product Ratings'
    
    low_ratings = product_ratings.loc[product_ratings['Rating']!='No Ratings']
    low_ratings['Rating'] = low_ratings['Rating'].astype(float)
    low_ratings = low_ratings.loc[low_ratings['Rating']<4.0]
    low_ratings['violation'] = 'Product Rating <4.0 Stars'
    
    product_rating_violations = pd.concat([no_ratings, low_ratings]).reset_index(drop=True)
    
    return product_rating_violations

**Relevant keywords in beginning of title (Brand, ...)**

In [13]:
# Is brand name in first x words?
df['keywords'] = pd.Series(df['Product Name'].str.split(' '))
df['top_keywords'] = df['keywords'].map(lambda x: x[:5])

# number missing brand name in first x words
num_missing_brand = df[df.apply(lambda x: x['Brand'] in x['keywords'], axis=1)]
num_missing_brand_top = df[df.apply(lambda x: x['Brand'] in x['top_keywords'], axis=1)]

missing_brand_count = df['ASIN'].count() - num_missing_brand['ASIN'].count()
missing_brand = missing_brand_count / df['ASIN'].count()
missing_brand = round(missing_brand*100, 2)

missing_brand_top_count = df['ASIN'].count() - num_missing_brand_top['ASIN'].count()
missing_brand_top = missing_brand_top_count / df['ASIN'].count()
missing_brand_top = round(missing_brand_top*100,2)

# appending to output
row = ['Title - Missing Brand', missing_brand_count, missing_brand]
row = pd.Series(row, index=output.columns)
output = output.append(row, ignore_index=True)

row = ['Title - Missing Brand in beggining of title', missing_brand_top_count, missing_brand_top]
row = pd.Series(row, index=output.columns)
output = output.append(row, ignore_index=True)


print(str(missing_brand) + '% of your products do not have your brand name in title at all.')
print(str(missing_brand_top) + '% of your products do not have your brand name in the beginning of the title.')

NameError: name 'output' is not defined